In [ ]:
'''
FIXED DATA CLEANING SOLUTION
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================
# LOAD DATA
# ==============================================

print("=" * 80)
print("LOADING DATA")
print("=" * 80)

faostat_path = "/content/FAOSTAT_data_en_12-30-2025 (2).csv"
maize_xlsx_path = "/content/Maize-Production-2012-2020-Combined (1).xlsx"

# Load datasets
faostat_df = pd.read_csv(faostat_path, encoding='utf-8')
maize_df = pd.read_excel(maize_xlsx_path)

print(f"FAOSTAT data shape: {faostat_df.shape}")
print(f"Maize data shape: {maize_df.shape}")

# ==============================================
# SIMPLIFIED CLEANING FUNCTION
# ==============================================

def clean_maize_data_simple(df):
    """
    Simplified and robust cleaning function
    """
    print("\n" + "=" * 80)
    print("CLEANING PROCESS")
    print("=" * 80)
    
    # Make a copy
    df_clean = df.copy()
    
    # Step 1: Remove the first two rows (metadata rows)
    # Row 0: Years (2012, 2013, etc.)
    # Row 1: Column types (Area, Production, Yield)
    # Data starts at row 2
    
    # Extract years from row 0
    years = []
    for i in range(1, len(df_clean.columns)):  # Skip first column
        val = df_clean.iloc[0, i]
        if pd.notna(val):
            try:
                years.append(int(float(str(val))))  # Convert to int
            except:
                pass
    
    # Get unique years
    years = sorted(list(set(years)))
    print(f"Years found: {years}")
    
    # Step 2: Extract data starting from row 2
    data_df = df_clean.iloc[2:].reset_index(drop=True)
    
    # Step 3: Fix the first column (County names)
    data_df = data_df.rename(columns={data_df.columns[0]: "County"})
    
    # Step 4: For each year, extract the 3 columns (Area, Production, Yield)
    # The pattern repeats every 3 columns starting from column 1
    
    all_county_data = []
    
    for i, year in enumerate(years):
        # Calculate column indices (pattern: County, then Area, Production, Yield for each year)
        start_col = 1 + (i * 3)
        
        if start_col + 2 < len(data_df.columns):  # Ensure we have 3 columns
            # Extract the 3 columns for this year
            year_data = data_df.iloc[:, [0, start_col, start_col + 1, start_col + 2]].copy()
            
            # Rename columns
            year_data.columns = ['County', f'Area_HA_{year}', f'Production_MT_{year}', f'Yield_MT_HA_{year}']
            
            # Add year column
            year_data['Year'] = year
            
            all_county_data.append(year_data)
    
    # Step 5: Combine all years
    if all_county_data:
        combined_df = pd.concat(all_county_data, ignore_index=True)
    else:
        # Alternative: Try direct reshaping
        print("Using alternative reshaping method...")
        combined_df = pd.DataFrame()
    
    # Step 6: Convert to long format (if we have combined data)
    if not combined_df.empty:
        # Melt to long format
        melted_dfs = []
        
        for year in years:
            if f'Area_HA_{year}' in combined_df.columns:
                temp_df = combined_df[['County', 'Year', 
                                       f'Area_HA_{year}', 
                                       f'Production_MT_{year}', 
                                       f'Yield_MT_HA_{year}']].copy()
                temp_df = temp_df.rename(columns={
                    f'Area_HA_{year}': 'Area_HA',
                    f'Production_MT_{year}': 'Production_MT',
                    f'Yield_MT_HA_{year}': 'Yield_MT_HA'
                })
                melted_dfs.append(temp_df)
        
        if melted_dfs:
            long_df = pd.concat(melted_dfs, ignore_index=True)
        else:
            long_df = pd.DataFrame()
    else:
        # Create long format directly from original data
        print("Creating long format from original structure...")
        long_records = []
        
        # We have 47 counties (rows 2-48)
        for row_idx in range(2, len(df_clean)):
            county = df_clean.iloc[row_idx, 0]
            
            # Process each year's triple of columns
            for i, year in enumerate(years):
                start_col = 1 + (i * 3)
                
                if start_col + 2 < len(df_clean.columns):
                    area = df_clean.iloc[row_idx, start_col]
                    production = df_clean.iloc[row_idx, start_col + 1]
                    yield_val = df_clean.iloc[row_idx, start_col + 2]
                    
                    # Convert to numeric safely
                    try:
                        area_num = float(area) if pd.notna(area) else np.nan
                        prod_num = float(production) if pd.notna(production) else np.nan
                        yield_num = float(yield_val) if pd.notna(yield_val) else np.nan
                    except:
                        area_num = np.nan
                        prod_num = np.nan
                        yield_num = np.nan
                    
                    if pd.notna(area_num) or pd.notna(prod_num) or pd.notna(yield_num):
                        long_records.append({
                            'County': county,
                            'Year': year,
                            'Area_HA': area_num,
                            'Production_MT': prod_num,
                            'Yield_MT_HA': yield_num
                        })
        
        long_df = pd.DataFrame(long_records)
    
    # Step 7: Clean the data
    if not long_df.empty:
        # Convert numeric columns
        numeric_cols = ['Area_HA', 'Production_MT', 'Yield_MT_HA']
        for col in numeric_cols:
            if col in long_df.columns:
                long_df[col] = pd.to_numeric(long_df[col], errors='coerce')
        
        # Remove rows where all numeric values are NaN
        long_df = long_df.dropna(subset=numeric_cols, how='all')
        
        # Reset index
        long_df = long_df.reset_index(drop=True)
    
    print(f"Cleaned long format shape: {long_df.shape if not long_df.empty else 'Empty'}")
    return long_df, years

# ==============================================
# ALTERNATIVE APPROACH - DIRECT CLEANING
# ==============================================

print("\n" + "=" * 80)
print("ALTERNATIVE DIRECT CLEANING")
print("=" * 80)

# Let's examine the structure more carefully
print("\nFirst few rows of raw data:")
print(maize_df.head(3))

# Create a clean dataframe directly
clean_data = []

# The data has this structure:
# Row 0: Years spread across columns
# Row 1: Column headers (Area, Production, Yield)
# Rows 2-48: County data

# Extract years from row 0
year_positions = {}
for col_idx in range(1, len(maize_df.columns)):  # Skip first column
    year_val = maize_df.iloc[0, col_idx]
    if pd.notna(year_val):
        try:
            year = int(year_val)
            # Find which columns belong to this year (next 3 columns)
            year_positions[year] = col_idx
        except:
            pass

print(f"\nYear positions found: {year_positions}")

# Process each county (rows 2 to end)
for row_idx in range(2, len(maize_df)):
    county = maize_df.iloc[row_idx, 0]
    
    for year, start_col in year_positions.items():
        # Each year has 3 columns: Area, Production, Yield
        if start_col + 2 < len(maize_df.columns):
            area = maize_df.iloc[row_idx, start_col]
            production = maize_df.iloc[row_idx, start_col + 1]
            yield_val = maize_df.iloc[row_idx, start_col + 2]
            
            # Skip if all values are NaN
            if pd.isna(area) and pd.isna(production) and pd.isna(yield_val):
                continue
            
            clean_data.append({
                'County': county,
                'Year': year,
                'Area_HA': area,
                'Production_MT': production,
                'Yield_MT_HA': yield_val
            })

# Create DataFrame
maize_clean = pd.DataFrame(clean_data)

# Convert numeric columns
for col in ['Area_HA', 'Production_MT', 'Yield_MT_HA']:
    maize_clean[col] = pd.to_numeric(maize_clean[col], errors='coerce')

print(f"\nCleaned data shape: {maize_clean.shape}")
print("\nFirst 10 rows of cleaned data:")
print(maize_clean.head(10))

# ==============================================
# DATA VALIDATION
# ==============================================

print("\n" + "=" * 80)
print("DATA VALIDATION")
print("=" * 80)

print(f"\nNumber of counties: {maize_clean['County'].nunique()}")
print(f"Years covered: {sorted(maize_clean['Year'].unique())}")
print(f"Total records: {len(maize_clean)}")

print("\nMissing values:")
print(maize_clean.isnull().sum())

print("\nBasic statistics:")
print(maize_clean.describe())

# Check data consistency
maize_clean['Yield_Calculated'] = maize_clean['Production_MT'] / maize_clean['Area_HA']
maize_clean['Yield_Difference'] = abs(maize_clean['Yield_MT_HA'] - maize_clean['Yield_Calculated'])

print(f"\nAverage yield difference: {maize_clean['Yield_Difference'].mean():.6f}")
print(f"Max yield difference: {maize_clean['Yield_Difference'].max():.6f}")

# ==============================================
# CREATE WIDE FORMAT
# ==============================================

print("\n" + "=" * 80)
print("CREATING WIDE FORMAT")
print("=" * 80)

# Pivot to wide format
wide_format = maize_clean.pivot_table(
    index='County',
    columns='Year',
    values=['Area_HA', 'Production_MT', 'Yield_MT_HA']
)

# Flatten column names
wide_format.columns = [f'{col[0]}_{col[1]}' for col in wide_format.columns]
wide_format = wide_format.reset_index()

print(f"Wide format shape: {wide_format.shape}")
print("\nWide format columns:")
print(list(wide_format.columns))
print("\nWide format (first 5 rows):")
print(wide_format.head())

# ==============================================
# SAVE CLEANED DATA
# ==============================================

print("\n" + "=" * 80)
print("SAVING CLEANED DATA")
print("=" * 80)

# Save both formats
maize_clean.to_csv('maize_kenya_clean_long.csv', index=False)
wide_format.to_csv('maize_kenya_clean_wide.csv', index=False)

print("✓ Long format saved: maize_kenya_clean_long.csv")
print("✓ Wide format saved: maize_kenya_clean_wide.csv")

# ==============================================
# EXPLORE FAOSTAT DATA
# ==============================================

print("\n" + "=" * 80)
print("FAOSTAT DATA EXPLORATION")
print("=" * 80)

print(f"\nFAOSTAT columns: {list(faostat_df.columns)}")
print("\nFAOSTAT data sample:")
print(faostat_df.head())

# Check for maize data in FAOSTAT
if 'Item' in faostat_df.columns:
    maize_faostat = faostat_df[faostat_df['Item'].str.contains('Maize', case=False, na=False)]
    print(f"\nMaize records in FAOSTAT: {len(maize_faostat)}")
    if len(maize_faostat) > 0:
        print("\nMaize data from FAOSTAT:")
        print(maize_faostat[['Year', 'Area', 'Value', 'Unit']].head())

# ==============================================
# VISUALIZATION
# ==============================================

print("\n" + "=" * 80)
print("CREATING VISUALIZATIONS")
print("=" * 80)

# Create some basic visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Yield distribution
axes[0, 0].hist(maize_clean['Yield_MT_HA'].dropna(), bins=20, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].set_xlabel('Yield (MT/HA)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Maize Yield in Kenya')
axes[0, 0].grid(True, alpha=0.3)

# 2. Yield by year
yearly_avg = maize_clean.groupby('Year')['Yield_MT_HA'].mean()
axes[0, 1].plot(yearly_avg.index, yearly_avg.values, marker='o', linewidth=2, color='green')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Average Yield (MT/HA)')
axes[0, 1].set_title('Average Maize Yield Trend 2012-2020')
axes[0, 1].grid(True, alpha=0.3)

# 3. Top 10 counties by yield
top_counties = maize_clean.groupby('County')['Yield_MT_HA'].mean().sort_values(ascending=False).head(10)
axes[1, 0].barh(range(len(top_counties)), top_counties.values, color='orange')
axes[1, 0].set_yticks(range(len(top_counties)))
axes[1, 0].set_yticklabels(top_counties.index)
axes[1, 0].set_xlabel('Average Yield (MT/HA)')
axes[1, 0].set_title('Top 10 Counties by Average Yield')
axes[1, 0].grid(True, alpha=0.3)

# 4. Area vs Yield scatter
axes[1, 1].scatter(maize_clean['Area_HA'], maize_clean['Yield_MT_HA'], alpha=0.5, color='purple')
axes[1, 1].set_xlabel('Area (HA)')
axes[1, 1].set_ylabel('Yield (MT/HA)')
axes[1, 1].set_title('Area vs Yield Relationship')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('maize_yield_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualizations saved as 'maize_yield_analysis.png'")

# ==============================================
# FINAL SUMMARY
# ==============================================

print("\n" + "=" * 80)
print("CLEANING PROCESS COMPLETE!")
print("=" * 80)

print(f"\nSUMMARY:")
print(f"• Original data: {maize_df.shape[0]} rows, {maize_df.shape[1]} columns")
print(f"• Cleaned data: {maize_clean.shape[0]} rows, {maize_clean.shape[1]} columns")
print(f"• Counties: {maize_clean['County'].nunique()}")
print(f"• Years: {sorted(maize_clean['Year'].unique())}")
print(f"• Time period: {maize_clean['Year'].min()} to {maize_clean['Year'].max()}")
print(f"• Average yield: {maize_clean['Yield_MT_HA'].mean():.2f} MT/HA")
print(f"• Total production (average): {maize_clean['Production_MT'].mean():,.0f} MT")

print("\n✓ Data is now cleaned and ready for machine learning!")
print("✓ Use 'maize_clean' DataFrame for modeling")
print("✓ Files saved: maize_kenya_clean_long.csv, maize_kenya_clean_wide.csv")